In [1]:
# -*- coding: utf-8 -*-
from functools import reduce
from quant_utils.performance import Performance
import data_functions.portfolio_data as pds
import data_functions.fund_data as dfs
from quant_utils.constant import DB_CONN_JJTG_DATA
import quant_utils.data_moudle as dm
from quant_utils.utils import cut_series_to_group, cal_series_rank
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import dataframe_image as dfi
from joblib import Parallel, delayed
plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号

In [11]:
def highlight_first_row(s):
    result_list = []
    for _ in range(len(s)):
        if s["PORTFOLIO_NAME"] in ["知己优选-现金宝", "知己优选-货币小福星"]:
            result_list.append('background-color: seashell; color: #a31939')
        else:
            result_list.append('color: #FFEFD5')
    return result_list

df = pds.query_portfolio_daily_performance("20231221")
df = df.reset_index(drop=True)
df1 = (
    df.style.format(
        {
            "近1日": "{:.4f}",
        },
        na_rep="--"
      )
      .apply(highlight_first_row, axis=1)
)
df1

,PORTFOLIO_NAME,LISTED_DATE,TO_CLIENT_DATE,近1日,近1周,近1月,近3月,近6月,YTD,近1年,成立日,对客日,最大回撤
0,知己优选-现金宝,2023-03-17,2023-03-17,0.0080,0.050330,0.190758,0.547241,1.079340,--,--,1.690732,1.690732,0.000000
1,知己优选-货币小福星,2022-07-22,2022-07-22,0.0081,0.061887,0.185779,0.556612,1.087374,2.308967,2.395340,3.013626,3.013626,0.033259
2,知己优选-短债增强,2023-11-22,2023-11-22,0.0098,0.096810,--,--,--,--,--,0.198479,0.198479,0.013791
3,知己优选-月月享,2021-10-15,2022-01-24,0.0765,-0.022765,-0.413607,-0.160454,0.083434,2.158686,2.656503,3.021194,1.114927,2.442616
4,知己优选-固收佳进取,2022-07-22,2022-07-22,0.1251,-0.182212,-0.905676,-0.732966,-0.902168,0.503589,1.183184,-0.870455,-0.870455,2.732979
5,知己优选-均衡小确幸,2021-09-03,2022-01-24,0.2394,-0.593297,-1.998521,-2.090874,-3.651886,-2.431036,-1.203942,-9.331404,-8.858162,13.563448
6,知己优选-进取全明星,2021-09-03,2022-01-24,0.4616,-1.304740,-3.974748,-3.545973,-8.668174,-10.206934,-8.577166,-25.843286,-24.713864,31.143562
7,知己优选-300增强,2023-11-22,2023-11-22,0.8119,-0.579841,--,--,--,--,--,-4.821130,-4.821130,6.073815
8,知己优选-500增强,2023-11-22,2023-11-22,0.4746,-2.187232,--,--,--,--,--,-3.204777,-3.204777,4.627161
9,知己优选-1000增强,2023-11-22,2023-11-22,0.4924,-2.710801,--,--,--,--,--,-3.608388,-3.608388,5.063979


In [12]:
type(df1)

pandas.io.formats.style.Styler

In [4]:
df

,PORTFOLIO_NAME,LISTED_DATE,TO_CLIENT_DATE,近1日,近1周,近1月,近3月,近6月,YTD,近1年,成立日,对客日,最大回撤
END_DATE,,,,,,,,,,,,,
2023-12-21,知己优选-现金宝,2023-03-17,2023-03-17,0.008050,0.050330,0.190758,0.547241,1.079340,NaN,NaN,1.690732,1.690732,0.000000
2023-12-21,知己优选-货币小福星,2022-07-22,2022-07-22,0.008076,0.061887,0.185779,0.556612,1.087374,2.308967,2.395340,3.013626,3.013626,0.033259
2023-12-21,知己优选-短债增强,2023-11-22,2023-11-22,0.009788,0.096810,NaN,NaN,NaN,NaN,NaN,0.198479,0.198479,0.013791
2023-12-21,知己优选-月月享,2021-10-15,2022-01-24,0.076483,-0.022765,-0.413607,-0.160454,0.083434,2.158686,2.656503,3.021194,1.114927,2.442616
2023-12-21,知己优选-固收佳进取,2022-07-22,2022-07-22,0.125063,-0.182212,-0.905676,-0.732966,-0.902168,0.503589,1.183184,-0.870455,-0.870455,2.732979
2023-12-21,知己优选-均衡小确幸,2021-09-03,2022-01-24,0.239390,-0.593297,-1.998521,-2.090874,-3.651886,-2.431036,-1.203942,-9.331404,-8.858162,13.563448
2023-12-21,知己优选-进取全明星,2021-09-03,2022-01-24,0.461565,-1.304740,-3.974748,-3.545973,-8.668174,-10.206934,-8.577166,-25.843286,-24.713864,31.143562
2023-12-21,知己优选-300增强,2023-11-22,2023-11-22,0.811887,-0.579841,NaN,NaN,NaN,NaN,NaN,-4.821130,-4.821130,6.073815
2023-12-21,知己优选-500增强,2023-11-22,2023-11-22,0.474642,-2.187232,NaN,NaN,NaN,NaN,NaN,-3.204777,-3.204777,4.627161


In [39]:
file_path = 'E:/excel/渣打.xlsx'
trade_dt = '20240506'
df = pd.read_excel(file_path, sheet_name="代销")
df["TICKER_SYMBOL"] = df["TICKER_SYMBOL"].astype("str")
df["TICKER_SYMBOL"] = df["TICKER_SYMBOL"].str.rjust(6,"0")
# df = (
#     df
#     .merge(dfs.query_fund_info())
#     .merge(
#         dfs.query_fund_performance_rank_pct(trade_dt)[
#             ["TICKER_SYMBOL", "LEVEL","F_AVGRETURN_QUARTER", "F_AVGRETURN_HALFYEAR", "F_AVGRETURN_YEAR", "F_AVGRETURN_TWOYEAR"]
#         ], how="left"
#     )
# )
alpha = pd.read_excel("E:/excel/!基金筛选-基金超额(2Y).xlsx")
alpha["TICKER_SYMBOL"] = alpha["TICKER_SYMBOL"].astype("str")
alpha["TICKER_SYMBOL"] = alpha["TICKER_SYMBOL"].str.rjust(6,"0")

In [46]:
df1 = df.merge(alpha)
df1.query("TOTAL_SCORE_GROUP <= 3 and LEVEL_1 == '主动权益'")

,TICKER_SYMBOL,END_DATE,SEC_SHORT_NAME,MANAGER_NAME,net_asset,LEVEL_1,LEVEL_2,LEVEL_3,IS_MAIN,IF_MANAGER_CHANGE_6M,...,MANAGEMENT_FEE,TRUSTEE_FEE,SALE_FEE,TOTAL_FEE,7d,30d,if_in_tranche,tranche,FIRST_BUY,MANAGEMENT_FEE_INDIVIDUAL
5,202011,2024-04-30,南方优选价值混合A,罗安安,8.96,主动权益,A股主题,无主题,1,0,...,1.2,0.2,0.0,1.4,0.50,0.5,0.0,NaN,1.0,10.00
8,000527,2024-04-30,南方新优享灵活配置混合A,章晖,38.58,主动权益,A股主题,无主题,1,0,...,1.2,0.2,0.0,1.4,0.75,0.5,0.0,NaN,1.0,40.00
11,160106,2024-04-30,南方高增长混合(LOF),张延闽,13.36,主动权益,A股主题,无主题,1,0,...,1.2,0.2,0.0,1.4,0.50,0.5,0.0,NaN,1.0,10.00
12,202023,2024-04-30,南方优选成长混合A,骆帅,29.17,主动权益,A股主题,无主题,1,0,...,1.2,0.2,0.0,1.4,0.50,0.5,0.0,NaN,1.0,10.00
13,160133,2024-04-30,南方天元新产业股票(LOF),蒋秋洁,10.02,主动权益,A股主题,无主题,1,0,...,1.2,0.2,0.0,1.4,0.75,0.5,0.0,NaN,1.0,10.00
14,003956,2024-04-30,南方产业智选股票,恽雷,5.15,主动权益,A股主题,无主题,1,0,...,1.2,0.2,0.0,1.4,0.75,0.5,0.0,NaN,1.0,10.00
16,202001,2024-04-30,南方稳健成长混合,应帅,15.59,主动权益,A股主题,无主题,1,0,...,1.2,0.2,0.0,1.4,0.50,0.5,0.0,NaN,1.0,0.00
18,001536,2024-04-30,南方君选灵活配置混合,卢玉珊,2.57,主动权益,A股主题,无主题,1,0,...,1.2,0.2,0.0,1.4,0.75,0.5,0.0,NaN,1.0,20.00
19,001181,2024-04-30,南方改革机遇灵活配置混合,卢玉珊,3.87,主动权益,A股主题,无主题,1,0,...,1.2,0.2,0.0,1.4,0.75,0.5,1.0,优选池,1.0,20.00
21,008264,2024-04-30,南方ESG主题股票A,章晖,5.08,主动权益,A股主题,无主题,1,0,...,1.2,0.2,0.0,1.4,0.75,0.5,0.0,NaN,1.0,50.00


UndefinedVariableError: name 'TOTAL_SCORE_GROUP' is not defined